<a href="https://colab.research.google.com/github/ysikalie/colorization/blob/master/full_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

import tensorflow as tf
import keras
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.preprocessing import image
from keras.engine import Layer
from keras.applications.inception_resnet_v2 import preprocess_input
from keras.layers import Conv2D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate,Dense
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard 
from keras.models import Sequential, Model
from keras.layers.core import RepeatVector, Permute
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

In [42]:
!python --version

Python 2.7.15rc1


In [0]:
# !tar  xvJf Train.tar.xz

In [0]:
# Get images
X = []
for filename in os.listdir('Train/'):
    X.append(img_to_array(load_img('Train/'+filename)))
X = np.array(X, dtype=float)
Xtrain = 1.0/255*X

#Addition
X_test=[]
for filename in os.listdir('Test/'):
    X_test.append(img_to_array(load_img('Test/'+filename)))
X_test = np.array(X_test, dtype=float)
Xtest = 1.0/255*X_test

#Load weights
inception = InceptionResNetV2(weights='imagenet', include_top=True)
inception.summary()

inception.graph = tf.get_default_graph()

In [0]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

model = VGG16(weights='imagenet', include_top=True)

img_path = '1.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

features = model.predict(x)
model.summary()

In [43]:
# embed_input = Input(shape=(1000,))

#Mid-Level features network
gray_encoder_input = Input(shape=(256, 256, 1,))
gray_encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_input)
gray_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(gray_encoder_output)
gray_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(gray_encoder_output)
gray_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(gray_encoder_output)


#Add
#Global Features network
color_encoder_input = Input(shape=(256, 256, 3,))
color_encoder_output = Conv2D(64, (3,3), activation='relu', padding='same', strides=2)(color_encoder_input)
color_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(color_encoder_output)
color_encoder_output = Conv2D(128, (3,3), activation='relu', padding='same', strides=2)(color_encoder_output)
color_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same')(color_encoder_output)
color_encoder_output = Conv2D(256, (3,3), activation='relu', padding='same', strides=2)(color_encoder_output)
color_encoder_output = Conv2D(512, (3,3), activation='relu', padding='same')(color_encoder_output)
color_encoder_output = Flatten(name='flatten')(color_encoder_output)
color_encoder_output = Dense(1024, activation='sigmoid')(color_encoder_output)
color_encoder_output = Dense(512, activation='sigmoid')(color_encoder_output)
color_encoder_output = Dense(256, activation='sigmoid')(color_encoder_output)

#Fusion
fusion_output = RepeatVector(32 * 32)(color_encoder_output) 
fusion_output = Reshape(([32, 32, 256]))(fusion_output)
fusion_output = concatenate([gray_encoder_output, fusion_output], axis=3) 
 

#Decoder
decoder_output = Conv2D(128, (3,3), activation='relu', padding='same')(fusion_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(64, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)
decoder_output = Conv2D(32, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(16, (3,3), activation='relu', padding='same')(decoder_output)
decoder_output = Conv2D(2, (3, 3), activation='tanh', padding='same')(decoder_output)
decoder_output = UpSampling2D((2, 2))(decoder_output)

model = Model(inputs=[gray_encoder_input, color_encoder_input], outputs=decoder_output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_44 (InputLayer)           (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
conv2d_458 (Conv2D)             (None, 128, 128, 64) 1792        input_44[0][0]                   
__________________________________________________________________________________________________
conv2d_459 (Conv2D)             (None, 128, 128, 128 73856       conv2d_458[0][0]                 
__________________________________________________________________________________________________
conv2d_460 (Conv2D)             (None, 64, 64, 128)  147584      conv2d_459[0][0]                 
__________________________________________________________________________________________________
input_43 (

In [0]:
def create_inception_embedding(grayscaled_rgb):
    grayscaled_rgb_resized = []
    for i in grayscaled_rgb:
        i = resize(i, (299, 299, 3), mode='constant')
        grayscaled_rgb_resized.append(i)
    grayscaled_rgb_resized = np.array(grayscaled_rgb_resized)
    grayscaled_rgb_resized = preprocess_input(grayscaled_rgb_resized)
    with inception.graph.as_default():
        embed = inception.predict(grayscaled_rgb_resized)
    return embed

# Image transformer
datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        rotation_range=20,
        horizontal_flip=True)

#Generate training data
batch_size = 1.

def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        print([X_batch, embed], Y_batch)
        yield ([X_batch, embed], Y_batch)

#addition
def testImage_a_b_gen(batch_size):
    for batch in Xtest:
        grayscaled_rgb = gray2rgb(rgb2gray(batch))
        embed = create_inception_embedding(grayscaled_rgb)
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        X_batch = X_batch.reshape(X_batch.shape+(1,))
        Y_batch = lab_batch[:,:,:,1:] / 128
        print([X_batch, embed], Y_batch)
        yield ([X_batch, embed], Y_batch)


#Train model
import tensorflow as tf
from keras.optimizers import RMSprop
rmsprop = RMSprop(lr=1e-4)
model.compile(optimizer=rmsprop, loss='mse')
filepath = 'checkpoint'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,
                            monitor='val_acc',
                            verbose=1,
                            save_best_only=True,
                            mode='max')
model.fit_generator(image_a_b_gen(batch_size), epochs=1, 
                    steps_per_epoch=1,callbacks=[checkpoint],
                    validation_data=testImage_a_b_gen(batch_size),
                    validation_steps=10
                   )

Epoch 1/1


TypeError: ignored

In [0]:
！ nvidia-smi